<a href="https://colab.research.google.com/github/konamilk/mlb-player-digital-engagement-forecasting/blob/main/mlb_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sys
from pathlib import Path
if 'google.colab' in sys.modules:
    INPUT = Path('/content/input/')
elif 'kaggle_web_client' in sys.modules:
    INPUT = Path('../input/mlb-player-digital-engagement-forecasting')

In [3]:
if 'google.colab' in sys.modules:
  !mkdir ~/.kaggle
  !cp /content/drive/MyDrive/.kaggle/kaggle.json ~/.kaggle
  !chmod 600 ~/.kaggle/kaggle.json
  !pip install kaggle > /dev/null

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [4]:
if 'google.colab' in sys.modules:
    !pip install git+https://github.com/pfnet-research/xfeat.git > /dev/null
else:
    !pip install ../input/xfeatwhl/xfeat-0.1.1-py3-none-any.whl > /dev/null

  Running command git clone -q https://github.com/pfnet-research/xfeat.git /tmp/pip-req-build-2yka4ci0


In [5]:
if 'google.colab' in sys.modules:
  !kaggle competitions download -c mlb-player-digital-engagement-forecasting > /dev/null
  !mkdir input
  !unzip -o '*.zip' -d ./input/
  !rm *.zip
  !mv *.csv ./input/

100% 3.68k/3.68k [00:00<00:00, 1.82MB/s]
100% 173k/173k [00:00<00:00, 42.6MB/s]
100% 470M/470M [00:03<00:00, 141MB/s]
100% 820k/820k [00:00<00:00, 112MB/s]
100% 824/824 [00:00<00:00, 2.63MB/s]
100% 3.96M/3.96M [00:00<00:00, 130MB/s]
100% 191k/191k [00:00<00:00, 178MB/s]
mkdir: cannot create directory ‘input’: File exists
Archive:  train.csv.zip
  inflating: ./input/train.csv       

Archive:  example_test.csv.zip
  inflating: ./input/example_test.csv  

2 archives were successfully processed.


In [6]:
import numpy as np
import pandas as pd
import gc
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.functional as F
from torch.utils import data

In [7]:
TRAIN = INPUT/'train.csv'
TEAMS = INPUT/'awards.csv'
PLAYERS = INPUT/'players.csv'
AWARDS = INPUT/'awards.csv'
SEASONS = INPUT/'seasons.csv'

In [8]:
df = pd.read_csv(TRAIN)

In [9]:
def unpack_json(json_str):
    return np.nan if pd.isna(json_str) else pd.read_json(json_str)

In [10]:
import datetime
def next_date_as_int(date_as_int: int):
  dd = date_as_int % 100
  date_as_int = date_as_int // 100
  mm = date_as_int % 100
  yyyy = date_as_int // 100
  dt = datetime.datetime(yyyy, mm, dd)
  next_dt = dt + datetime.timedelta(days=1)
  return next_dt.year * 10000 + next_dt.month * 100 + next_dt.day


def int_date(dt: datetime):
  return dt.year * 10000 + dt.month * 100 + dt.day

In [11]:
class TrainDataset(object):
  def __init__(self, df_train, batch_size=5):
    self.df_train = df_train
    self.current = 0
    self.batch_size = batch_size
    df_player = pd.read_csv(PLAYERS)
    df_player[df_player['playerForTestSetAndFuturePreds'] == True]
    self.playerId =  df_player['playerId']
  
  def __iter__(self):
    return self

  def __next__(self):
    start = self.current * self.batch_size
    end = (self.current + 1) * self.batch_size
    self.current += 1

    if start >= self.df_train.shape[0]:
      raise StopIteration()
    
    dates = self.df_train[start:end].date.unique()

    temps = []
    for date in dates:
      df_temp = pd.DataFrame(self.playerId)
      df_temp['date'] = date
      df_temp['date_playerId'] = str(next_date_as_int(date)) + '_' + df_temp['playerId'].astype(str)
      df_temp['target1'] = 0.0
      df_temp['target2'] = 0.0
      df_temp['target3'] = 0.0
      df_temp['target4'] = 0.0
      df_temp = df_temp.drop(['playerId'], axis=1)
      temps.append(df_temp)

    return  self.df_train[start:end].drop('nextDayPlayerEngagement', axis=1).set_index('date'), pd.concat(temps, axis=0).set_index('date'), self.df_train[start:end].nextDayPlayerEngagement

In [12]:
from xfeat import SelectCategorical, LabelEncoder, Pipeline, ConcatCombination, SelectNumerical, \
    ArithmeticCombinations, TargetEncoder, aggregation, GBDTFeatureSelector, GBDTFeatureExplorer

class PreProcessor(object):
  def __init__(self):
    self.label_encoder = Pipeline([
      SelectCategorical(exclude_cols=['engagementMetricsDate']),
      LabelEncoder(output_suffix=""),
      ])
    self.df_rosters = pd.DataFrame()

  def pre_process(self, df_dairy: pd.DataFrame, df_submission: pd.DataFrame, phase = "train") -> pd.DataFrame:

    X = df_submission.drop(['target1', 'target2', 'target3', 'target4'], axis=1)
    X['today'] = pd.to_datetime(X.index.to_numpy().astype(str))
    X['year'] = X['today'].dt.year
    X['month'] = X['today'].dt.month
    X['day'] = X['today'].dt.day
    X['dayofweek'] = X['today'].dt.dayofweek
    X['engagementMetricsDate'] = pd.to_datetime(X['date_playerId'].str.split('_', expand = True)[0])
    X['playerId'] = X['date_playerId'].str.split('_', expand = True)[1].astype(int)

    # join rosters

    if df_dairy['rosters'].notnull().sum() > 0:
      df_rosters = pd.concat(map(lambda x: unpack_json(x), df_dairy[df_dairy['rosters'].notnull()]['rosters']), axis=0)
      df_rosters['gameDate'] = pd.to_datetime(df_rosters['gameDate'])
      self.df_rosters = pd.concat([self.df_rosters, df_rosters],axis=0)
      self.df_rosters = self.df_rosters.drop_duplicates()
    X = pd.merge(X, self.df_rosters, how='left', left_on=['today', 'playerId'], right_on=['gameDate', 'playerId']).drop(['status'], axis=1)

    # # join player twitter followers
    # df_playerTwitterFollowers = pd.concat(map(lambda x: unpack_json(x),df_dairy[df_dairy['playerTwitterFollowers'].notnull()]['playerTwitterFollowers']),axis=0)
    # df_playerTwitterFollowers = df_playerTwitterFollowers[['date', 'playerId', 'numberOfFollowers']]
    # df_playerTwitterFollowers['year'] = df_playerTwitterFollowers['date'].dt.year
    # df_playerTwitterFollowers['month'] = df_playerTwitterFollowers['date'].dt.month
    # df_playerTwitterFollowers = df_playerTwitterFollowers.drop('date', axis=1)
    # X = pd.merge(X, df_playerTwitterFollowers, how='left', on=['playerId', 'year', 'month'])
    # del df_playerTwitterFollowers

    # # join team twitter followers
    # df_teamTwitterFollowers = pd.concat(map(lambda x: unpack_json(x),df_dairy[df_dairy['teamTwitterFollowers'].notnull()]['teamTwitterFollowers']),axis=0)
    # df_teamTwitterFollowers = df_teamTwitterFollowers[['date', 'teamId', 'numberOfFollowers']]
    # df_teamTwitterFollowers['year'] = df_teamTwitterFollowers['date'].dt.year
    # df_teamTwitterFollowers['month'] = df_teamTwitterFollowers['date'].dt.month
    # df_teamTwitterFollowers = df_teamTwitterFollowers.drop('date', axis=1)
    # X = pd.merge(X, df_teamTwitterFollowers, how='left', on=['teamId', 'year', 'month'])
    # del df_teamTwitterFollowers

    # # join games
    # df_games = pd.concat(map(lambda x: unpack_json(x),df_dairy[df_dairy['games'].notnull()]['games']),axis=0)
    # new_columns = ['gamePk', 'gameType', 'season', 'gameDate', 'gameTimeUTC', 'resumeDate',
    #       'resumedFrom', 'codedGameState', 'detailedGameState', 'isTie',
    #       'gameNumber', 'doubleHeader', 'dayNight', 'scheduledInnings',
    #       'gamesInSeries', 'seriesDescription', 'teamId', 'teamName',
    #       'teamAbbrev', 'teamWins', 'teamLosses', 'teamWinPct', 'teamWinner',
    #       'teamScore', 'opponentId', 'opponentName', 'opponentAbbrev', 'opponentWins',
    #       'opponentLosses', 'opponentWinPct', 'opponentWinner', 'opponentScore']
    # df_games_home = df_games.copy()
    # df_games_home.columns = new_columns
    # df_games_away = df_games.copy()[['gamePk', 'gameType', 'season', 'gameDate', 'gameTimeUTC', 'resumeDate',
    #       'resumedFrom', 'codedGameState', 'detailedGameState', 'isTie',
    #       'gameNumber', 'doubleHeader', 'dayNight', 'scheduledInnings',
    #       'gamesInSeries', 'seriesDescription', 'awayId', 'awayName', 'awayAbbrev', 'awayWins',
    #       'awayLosses', 'awayWinPct', 'awayWinner', 'awayScore', 'homeId', 'homeName',
    #       'homeAbbrev', 'homeWins', 'homeLosses', 'homeWinPct', 'homeWinner',
    #       'homeScore']]
    # df_games_away.columns = new_columns
    # df_games_2 = pd.concat([df_games_home, df_games_away], axis=0).drop(['teamName', 'teamAbbrev', 'opponentName', 'opponentAbbrev', 'opponentWinner'], axis=1)
    # df_games_2['date'] = pd.to_datetime(df_games_2['gameDate'])
    # X = pd.merge(X, df_games_2, how='left', on=['teamId', 'date'])
    # del new_columns, df_games, df_games_home, df_games_away, df_games_2


    # label encoding
    if phase == 'train':
      X = X.drop(['today', 'date_playerId'], axis=1)
      encoded_X = self.label_encoder.fit_transform(X)
      X = pd.concat([X['engagementMetricsDate'], encoded_X, SelectNumerical().fit_transform(X)], axis=1)
    elif phase == 'eval':
      X = X.drop(['today', 'date_playerId'], axis=1)
      encoded_X = self.label_encoder.transform(X)
      X = pd.concat([X['engagementMetricsDate'], encoded_X, SelectNumerical().fit_transform(X)], axis=1)
    
    return X


In [13]:
# make Traing

df_train = df[df.date <= 20210331]

dts = TrainDataset(df_train, df_train.shape[0])
df_dairy, df_submission, targets = next(dts)
pre = PreProcessor()

X = pre.pre_process(df_dairy, df_submission)

# join targets
df_targets = pd.concat(map(lambda z: unpack_json(z), targets), axis=0)
df_targets['engagementMetricsDate'] = pd.to_datetime(df_targets['engagementMetricsDate'])
y = pd.merge(X[['engagementMetricsDate', 'playerId']], df_targets, on=['engagementMetricsDate', 'playerId'], how='left')
del df_targets



In [14]:
# make Validation
df_valid = df[(df.date >= 20210401) & (df.date <= 20210430)]
dts = TrainDataset(df_valid, df_valid.shape[0])

df_dairy, df_submission, targets = next(dts)


X_val = pre.pre_process(df_dairy, df_submission, phase='eval')
df_targets = pd.concat(map(lambda z: unpack_json(z), targets), axis=0)
df_targets['engagementMetricsDate'] = pd.to_datetime(df_targets['engagementMetricsDate'])
y_val = pd.merge(X_val[['engagementMetricsDate', 'playerId']], df_targets, on=['engagementMetricsDate', 'playerId'], how='left')

del df_targets

print(df_valid.shape, X_val.shape, y_val.shape)

(30, 12) (61830, 9) (61830, 6)


In [15]:
gc.collect()

103

In [16]:
import lightgbm as lgb

params = {'objective': 'regression',
             'metric': 'rmse',
             'verbose': -1,
             'feature_pre_filter': False,
            #  'lambda_l1': 1.9246603611247695,
            #  'lambda_l2': 0.0015207873611208637,
             'num_leaves': 45,
            #  'feature_fraction': 0.616,
             'bagging_fraction': 1.0,
             'bagging_freq': 0,
             'min_child_samples': 20,
}

target_columns = ['target1', 'target2', 'target3', 'target4']

models = []
for target_column in target_columns:
  lgb_train = lgb.Dataset(X.drop(['engagementMetricsDate'], axis=1), y[target_column])
  lgb_valid = lgb.Dataset(X_val.drop(['engagementMetricsDate'], axis=1), y_val[target_column])
  model = lgb.train(params, lgb_train, num_boost_round=200, valid_sets=lgb_valid, early_stopping_rounds=10)
  models.append(model)

[1]	valid_0's rmse: 4.6736
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's rmse: 4.67622
[3]	valid_0's rmse: 4.67898
[4]	valid_0's rmse: 4.64582
[5]	valid_0's rmse: 4.64816
[6]	valid_0's rmse: 4.61748
[7]	valid_0's rmse: 4.59811
[8]	valid_0's rmse: 4.59994
[9]	valid_0's rmse: 4.5902
[10]	valid_0's rmse: 4.59178
[11]	valid_0's rmse: 4.58359
[12]	valid_0's rmse: 4.58491
[13]	valid_0's rmse: 4.57446
[14]	valid_0's rmse: 4.57558
[15]	valid_0's rmse: 4.55465
[16]	valid_0's rmse: 4.55557
[17]	valid_0's rmse: 4.5564
[18]	valid_0's rmse: 4.5543
[19]	valid_0's rmse: 4.54914
[20]	valid_0's rmse: 4.54979
[21]	valid_0's rmse: 4.5458
[22]	valid_0's rmse: 4.54071
[23]	valid_0's rmse: 4.54192
[24]	valid_0's rmse: 4.53881
[25]	valid_0's rmse: 4.53761
[26]	valid_0's rmse: 4.53609
[27]	valid_0's rmse: 4.53274
[28]	valid_0's rmse: 4.53182
[29]	valid_0's rmse: 4.5305
[30]	valid_0's rmse: 4.52624
[31]	valid_0's rmse: 4.52657
[32]	valid_0's rmse: 4.5257
[33]	valid_0's rmse: 4.524

In [17]:

if 'google.colab' in sys.modules or 'kaggle_web_client' in sys.modules:
  df_test = df[(df.date >= 20210401) & (df.date <= 20210430)]
  iter_test = TrainDataset(df_valid, 5)
  for df_dairy, df_submission, _ in iter_test:
    X = pre.pre_process(df_dairy, df_submission, phase='eval')
    X = X.drop('engagementMetricsDate', axis=1)
    for (i, model) in enumerate(models):
      df_submission[target_columns[i]] = np.clip(model.predict(X), 0.0, 100.0)
    

In [18]:
if 'kaggle_secrets' in sys.modules:  # only run while on Kaggle
  import mlb

  env = mlb.make_env()
  iter_test = env.iter_test()

  for df_dairy, df_submission in iter_test:
    X = pre.pre_process(df_dairy, df_submission, phase='eval')
    X = X.drop('engagementMetricsDate', axis=1)

    for (i, model) in enumerate(models):
      df_submission[target_columns[i]] = np.clip(model.predict(X), 0.0, 100.0)

    env.predict(df_submission)